# Managing Data

In [2]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from random import shuffle,seed
from pprint import pprint

In [3]:
batch_size = 32
image_size = (32,32)
src_directory = '/home/shashank/Desktop/IIT_D ML/CNN/Classification_Assignment_Data/Data/'
dest_directory = '/home/shashank/Desktop/IIT_D ML/CNN/Classif_destination'

In [4]:
files=[]
for fx in os.listdir(src_directory):
    files.append(os.path.join(src_directory,fx))

pprint(files[:3])
seed(0)
shuffle(files)
print'-----------------'
pprint(files[:3])

['/home/shashank/Desktop/IIT_D ML/CNN/Classification_Assignment_Data/Data/cat.455.jpg',
 '/home/shashank/Desktop/IIT_D ML/CNN/Classification_Assignment_Data/Data/dog.1200.jpg',
 '/home/shashank/Desktop/IIT_D ML/CNN/Classification_Assignment_Data/Data/cat.765.jpg']
-----------------
['/home/shashank/Desktop/IIT_D ML/CNN/Classification_Assignment_Data/Data/dog.48.jpg',
 '/home/shashank/Desktop/IIT_D ML/CNN/Classification_Assignment_Data/Data/dog.137.jpg',
 '/home/shashank/Desktop/IIT_D ML/CNN/Classification_Assignment_Data/Data/dog.1297.jpg']


In [4]:
def load_image(fx):
    
    im = Image.open(fx)
    im = im.resize(image_size)
    im = np.array(im)
    im = im/255.0
    im = np.reshape(im,(1, image_size[0],image_size[1],3)) #doubt 
    return im

In [5]:
def load_labels(fx):
    label = os.path.basename(fx)
    label = label.split('.')[0]
    if label=="cat":
        return np.array([[1,0]])
    else:
        return np.array([[0,1]])

In [6]:
def create_batch(batch_number):
    batch = files[batch_number*batch_size:(batch_number+1)*batch_size]
    images = np.concatenate([load_image(f) for f in batch],axis=0)
    labels = np.concatenate([load_labels(f) for f in batch],axis=0)
    save_path = os.path.join(dest_directory,'Batch-{}.npz'.format(batch_number))
    np.savez(save_path,images=images,labels=labels)

In [7]:
no_files = len(files)
print 'total no. of files', no_files
no_batches = no_files/batch_size
print 'total no. of batches', no_batches

total no. of files 2880
total no. of batches 90


In [8]:
for ix in range(no_batches):
    create_batch(ix)

In [9]:
batch_path = os.path.join(dest_directory,"Batch-{}.npz")
train_batch_start_idx = 0
train_batch_end_idx = 70
val_batch_start_idx = 70
val_batch_end_idx = 80
test_batch_start_idx = 80
test_batch_end_idx=90


In [10]:
def Weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

In [11]:
def Bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [12]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [13]:
x_pl = tf.placeholder(tf.float32,shape=[None,32,32,3])
y_pl = tf.placeholder(tf.float32,shape=[None,2])

In [14]:
W_conv1 = Weight_variable([5,5,3,32])
B_conv1 = Bias_variable([32])

In [15]:
h_conv1 = tf.nn.relu(conv2d(x_pl,W_conv1)+B_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [16]:
W_conv2 = Weight_variable([5, 5, 32, 64])
B_conv2 = Bias_variable([64])

In [5]:
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+B_conv2)
h_pool2 = max_pool_2x2(h_conv2)



NameError: name 'conv2d' is not defined

In [18]:
W_fc1 = Weight_variable([8 * 8 * 64, 128])
B_fc1 = Bias_variable([128])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + B_fc1)

In [19]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [20]:
W_fc2 = Weight_variable([128, 2])
B_fc2 = Bias_variable([2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + B_fc2

In [21]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_pl, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [22]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_pl, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [23]:
max_epochs = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for ex in range(max_epochs):
        print 'epoch number:',ex
        print'-------------------------------------------'
        for train_idx in range(train_batch_start_idx, train_batch_end_idx):
            train = np.load(batch_path.format(train_idx))
            train_step.run(feed_dict={x_pl: train["images"], y_pl : train["labels"], keep_prob : 0.5})
            
        train_accuracy = 0.0
        for train_idx in range(train_batch_start_idx, train_batch_end_idx):
            train = np.load(batch_path.format(train_idx))
            train_accuracy += accuracy.eval(feed_dict={x_pl: train["images"],y_pl:train["labels"],keep_prob : 1.0  })
        train_accuracy /= (train_batch_end_idx - train_batch_start_idx)
        print 'train accuracy is :',train_accuracy*100
        
        
        test_accuracy = 0.0
        for test_idx in range(test_batch_start_idx, test_batch_end_idx):
            test = np.load(batch_path.format(test_idx))
            test_accuracy += accuracy.eval(feed_dict={x_pl: test["images"],y_pl:test["labels"],keep_prob: 1.0  })
        test_accuracy /= (test_batch_end_idx - test_batch_start_idx)
        print 'testing accuracy',test_accuracy*100
        print'--------------------------------------------'

epoch number: 0
-------------------------------------------
train accuracy is : 43.75
testing accuracy 57.5
--------------------------------------------
epoch number: 1
-------------------------------------------
train accuracy is : 50.0
testing accuracy 58.4375
--------------------------------------------
epoch number: 2
-------------------------------------------
train accuracy is : 71.875
testing accuracy 57.5
--------------------------------------------
epoch number: 3
-------------------------------------------
train accuracy is : 68.75
testing accuracy 56.875
--------------------------------------------
epoch number: 4
-------------------------------------------
train accuracy is : 71.875
testing accuracy 61.5625
--------------------------------------------
epoch number: 5
-------------------------------------------
train accuracy is : 71.875
testing accuracy 55.0
--------------------------------------------
epoch number: 6
-------------------------------------------
train accura